In [6]:
import gymnasium as gym
env = gym.make('Humanoid-v4')


# required before you can step the environment
env.reset()

# sample action:
print("sample action:", env.action_space.sample())

# observation space shape:
print("observation space shape:", env.observation_space.shape)

# sample observation:
print("sample observation:", env.observation_space.sample())

env.close()

sample action: [-0.0262878  -0.0707781   0.01077491  0.18597013 -0.28866938 -0.1927571
 -0.00061035  0.02415133 -0.28179666  0.10531954 -0.09529556  0.0039074
 -0.20114999 -0.3513349   0.35544717  0.2202607   0.00957036]
observation space shape: (376,)
sample observation: [ 1.65072801e+00  3.49824275e-01  1.07267308e+00  8.08914985e-01
  1.14453335e+00 -2.69387142e-01 -9.47386867e-01 -7.04842441e-01
 -6.68760830e-01 -1.98920333e+00 -1.60072527e+00 -5.50701367e-01
 -1.22791629e-02 -2.37538133e-01  7.24179568e-01  2.40891525e+00
  1.34889758e-01  5.60877095e-02  8.46617293e-01  2.25126016e-01
 -6.78857212e-01 -6.84916662e-01 -1.39690270e+00 -1.22267522e-02
  1.06797189e+00 -3.47251665e-01 -4.84625659e-01 -5.21622363e-01
 -5.50151455e-01 -1.44240230e+00 -4.01907682e-01  1.24298661e+00
  3.09565818e+00  4.72289627e-01  1.10599558e+00 -1.21698371e+00
  3.79472439e-01 -2.44261220e-01 -3.92637960e-01  6.39849241e-01
  1.42595433e-01  7.36241131e-01 -1.44940498e+00 -9.44411483e-03
  2.59932213

# Humanoid Standup

In [7]:
import numpy as np
import gymnasium as gym
from gymnasium import utils
from gymnasium.envs.mujoco import MuJocoPyEnv
from gymnasium.spaces import Box


class HumanoidStandupEnv(MuJocoPyEnv, utils.EzPickle):
    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
        "render_fps": 67,
    }

    def __init__(self, **kwargs):
        observation_space = Box(
            low=-np.inf, high=np.inf, shape=(376,), dtype=np.float64
        )
        MuJocoPyEnv.__init__(
            self,
            "humanoidstandup.xml",
            5,
            observation_space=observation_space,
            **kwargs
        )
        utils.EzPickle.__init__(self, **kwargs)

    def _get_obs(self):
        data = self.sim.data
        return np.concatenate(
            [
                data.qpos.flat[2:],
                data.qvel.flat,
                data.cinert.flat,
                data.cvel.flat,
                data.qfrc_actuator.flat,
                data.cfrc_ext.flat,
            ]
        )

    def step(self, a):
        self.do_simulation(a, self.frame_skip)
        pos_after = self.sim.data.qpos[2]
        data = self.sim.data
        uph_cost = (pos_after - 0) / self.model.opt.timestep

        quad_ctrl_cost = 0.1 * np.square(data.ctrl).sum()
        quad_impact_cost = 0.5e-6 * np.square(data.cfrc_ext).sum()
        quad_impact_cost = min(quad_impact_cost, 10)
        reward = uph_cost - quad_ctrl_cost - quad_impact_cost + 1

        if self.render_mode == "human":
            self.render()
        return (
            self._get_obs(),
            reward,
            False,
            False,
            dict(
                reward_linup=uph_cost,
                reward_quadctrl=-quad_ctrl_cost,
                reward_impact=-quad_impact_cost,
            ),
        )

    def reset_model(self):
        c = 0.01
        self.set_state(
            self.init_qpos + self.np_random.uniform(low=-c, high=c, size=self.model.nq),
            self.init_qvel
            + self.np_random.uniform(
                low=-c,
                high=c,
                size=self.model.nv,
            ),
        )
        return self._get_obs()

    def viewer_setup(self):
        assert self.viewer is not None
        self.viewer.cam.trackbodyid = 1
        self.viewer.cam.distance = self.model.stat.extent * 1.0
        self.viewer.cam.lookat[2] = 0.8925
        self.viewer.cam.elevation = -20

ModuleNotFoundError: No module named 'gym'